In this file the data will be merged into on table for further processing.
This will be done by connecting with the database and pass SQL querries through the connection.

At first, the required modules are imported:

In [2]:
import pandas as pd
import json
import mysql.connector
import os

Next, the credentials are loaded and the connection to the database is etablished:

In [20]:
#get credentials for MySQL/dbms
with open(r'C:\Users\mathi\Desktop\Datenanalyse\Credentials\mysql.json') as json_file:
    db_credentials = json.load(json_file)['MySQL']

#credentials as variables
host='localhost'
user=db_credentials['user']
password=db_credentials['password']
database='feederwatch'

#etablish connection to the dbms
mydb = mysql.connector.connect(
  host=host,
  user=user,
  password=password)

mycursor = mydb.cursor()

#select database
query=f'USE {database}'
mycursor.execute(query)

Now it's controlled if all the tables are availible and how they are labled.

In [3]:
query='SHOW TABLES'
mycursor.execute(query)
for table in mycursor:
    print(table)

('pfw_1988_1995_public',)
('pfw_1996_2000_public',)
('pfw_2001_2005_public',)
('pfw_2006_2010_public',)
('pfw_2011_2015_public',)
('pfw_2016_2020_public',)
('pfw_2021_public',)
('pfw_count_site_data_public_2021',)


Because the names are just adopted from the original files, they are renamed into a shorter version:

In [4]:
query='''RENAME TABLE
        pfw_1988_1995_public TO pfw_1988,
        pfw_1996_2000_public TO pfw_1996,
        pfw_2001_2005_public TO pfw_2001,
        pfw_2006_2010_public TO pfw_2006,
        pfw_2011_2015_public TO pfw_2011,
        pfw_2016_2020_public TO pfw_2016,
        pfw_2021_public TO pfw_2021,
        pfw_count_site_data_public_2021 TO pfw_site_data'''

mycursor.execute(query)

query='SHOW TABLES'
mycursor.execute(query)
for table in mycursor:
    print(table)

('pfw_1988',)
('pfw_1996',)
('pfw_2001',)
('pfw_2006',)
('pfw_2011',)
('pfw_2016',)
('pfw_2021',)
('pfw_site_data',)


Because it's expected that tables with data from different years have the same structure, they are grouped together to acess them easily with loops later on. 

In [4]:
observation_tables=set()
query='SHOW TABLES'
mycursor.execute(query)
for table in mycursor:
    observation_tables.add(table[0])

observation_tables.discard('pfw_site_data') #removes site_data from set because it should be treated differently

print(observation_tables)

{'pfw_2016', 'observations', 'pfw_1988', 'pfw_2001', 'pfw_2021', 'pfw_2006', 'pfw_2011', 'pfw_1996'}


The tables containing the observations should look all the same, and they contain the observations timestamp, they can be merged into a bigger one. Before this step is performed, it should be ensured that the tables all have the same column names and order.

In [6]:
column_names=pd.DataFrame()

for table in observation_tables:
    query=f'DESCRIBE {table}'
    mycursor.execute(query)
    columns=[]
    for line in mycursor:
        columns.append(line[0])
    
    temp_df=pd.DataFrame({table: columns})
    column_names=pd.concat([column_names, temp_df], axis=1)

print(column_names)

              pfw_1988            pfw_2001            pfw_2011  \
0                index               index               index   
1               LOC_ID              LOC_ID              LOC_ID   
2             LATITUDE            LATITUDE            LATITUDE   
3            LONGITUDE           LONGITUDE           LONGITUDE   
4    SUBNATIONAL1_CODE   SUBNATIONAL1_CODE   SUBNATIONAL1_CODE   
5      ENTRY_TECHNIQUE     ENTRY_TECHNIQUE     ENTRY_TECHNIQUE   
6               SUB_ID              SUB_ID              SUB_ID   
7               OBS_ID              OBS_ID              OBS_ID   
8                Month               Month               Month   
9                  Day                 Day                 Day   
10                Year                Year                Year   
11      PROJ_PERIOD_ID      PROJ_PERIOD_ID      PROJ_PERIOD_ID   
12        SPECIES_CODE        SPECIES_CODE        SPECIES_CODE   
13            HOW_MANY            HOW_MANY            HOW_MANY   
14        

Because the output exceeds the size limit of a jupyter notebook cell (at least in VS Code), visual inspection dosn't work. 
Instead, a boolen matrix can be generated, where all column names are compared with each other.

In [7]:
as_np_array = column_names.values
comparison_matrix = pd.DataFrame((as_np_array.T[:, None] == as_np_array.T).all(-1), column_names.columns, column_names.columns)
print(comparison_matrix)

          pfw_1988  pfw_2001  pfw_2011  pfw_2006  pfw_1996  pfw_2016  pfw_2021
pfw_1988      True      True      True      True      True      True     False
pfw_2001      True      True      True      True      True      True     False
pfw_2011      True      True      True      True      True      True     False
pfw_2006      True      True      True      True      True      True     False
pfw_1996      True      True      True      True      True      True     False
pfw_2016      True      True      True      True      True      True     False
pfw_2021     False     False     False     False     False     False     False


The comparison shows, that all tables have the same column names except 'pfw_2021'. For further investigation, the column names of 'pfw_2021' are displayed next to the names from another table.

In [8]:
column_names[['pfw_2021', 'pfw_1988']]

,pfw_2021,pfw_1988
0,index,index
1,loc_id,LOC_ID
2,latitude,LATITUDE
3,longitude,LONGITUDE
4,subnational1_code,SUBNATIONAL1_CODE
5,entry_technique,ENTRY_TECHNIQUE
6,sub_id,SUB_ID
7,obs_id,OBS_ID
8,Month,Month
9,Day,Day


A obvious differnce is, that they are written in differnt cases (lower/upper case). But because the case dosn't matter for merging the tables, this difference can be ignored. 
Another difference is visible in row 14, where one column is labeled 'valid' and the other 'PLUS_CODE'. It seems, that the column 'plus_code' is missing in 'pfw_2021', which leads to a shift and missmatch in all following columns and a NaN column at the end to fill up the data.
To figure out, what the missing column 'plus_code' should ontain, the FeederWatch_Data_Dictonary.xlsx, provided by Project FeederWatch, was checked. 
According to this documentation it should contain a binary value:

>"Variable indicating if the total number of a species seen was larger than the value reported".

Even though this variable dosn't seem to be very interesting for analysis, it can be keept, and an empty column can be created for the table, where it's missing.  

In [9]:
query='''ALTER TABLE pfw_2021
        ADD COLUMN plus_code BINARY NULL
        AFTER how_many'''
mycursor.execute(query)

To confirm the change, code from above is run again. 

In [10]:
column_names=pd.DataFrame()

for table in observation_tables:
    query=f'DESCRIBE {table}'
    mycursor.execute(query)
    columns=[]
    for line in mycursor:
        columns.append(line[0])
    
    temp_df=pd.DataFrame({table: columns})
    column_names=pd.concat([column_names, temp_df], axis=1)

column_names[['pfw_2021', 'pfw_1988']]

,pfw_2021,pfw_1988
0,index,index
1,loc_id,LOC_ID
2,latitude,LATITUDE
3,longitude,LONGITUDE
4,subnational1_code,SUBNATIONAL1_CODE
5,entry_technique,ENTRY_TECHNIQUE
6,sub_id,SUB_ID
7,obs_id,OBS_ID
8,Month,Month
9,Day,Day


Because now all tables have the same number and order of columns, they can be merged into a bigger one.
For this reason a new, empty table is created, where all others will be merged into.

In [11]:
query='CREATE TABLE IF NOT EXISTS observations LIKE pfw_1988'
mycursor.execute(query)

Because of the size of the tables, a normal insert query might create an DatabaseError (1206 (HY000): The total number of locks exceeds the lock table size). 
This can be avoided by inserting small chunks. The easiest way to do this, is by using an statement like this:

>WHERE id> i
>ORDER BY id ASC
>LIMIT chunkSize
>id=id+chunkSize

To run such a query an integer id is required. The tables have a column called OBS_ID, which can be ispected to check, if it can be used for such purpose.

In [13]:
query='SELECT OBS_ID FROM pfw_1988 ORDER BY OBS_ID LIMIT 10'
mycursor.execute(query)
for line in mycursor:
    print(line)

('OBS217571888',)
('OBS217571889',)
('OBS217572168',)
('OBS219656334',)
('OBS219657237',)
('OBS219657661',)
('OBS224435761',)
('OBS224436034',)
('OBS224452237',)
('OBS224452938',)


It turned out, that the OBS_ID increases, but the values are strings and a lot of numbers are missing inbetween.
To get a continues intiger, an auto incrementing primary key is created.
To avoid issues during insertion, the table observation dosn't get a key.

In [24]:
for table in observation_tables:
    query=f'ALTER TABLE {table} ADD ID INT NOT NULL AUTO_INCREMENT, ADD PRIMARY KEY (ID)'
    mycursor.execute(query)
    print(f'{table}: Primary Key added')

#and the observation table as well
table='observations'
query=f'ALTER TABLE {table} ADD ID INT NOT NULL'
mycursor.execute(query)
print(f'{table}: Column ID added')

pfw_1988: Primary Key added
pfw_2001: Primary Key added
pfw_2011: Primary Key added
pfw_2006: Primary Key added
pfw_1996: Primary Key added
pfw_2016: Primary Key added
pfw_2021: Primary Key added
observations: Column ID added


Now the new table can be populated by inserting the data from all other tables into it.
This is done chunk-wise to avoid memory issues, and in addition the default innodb_buffer_pool_size was increased.

In [21]:
chunk_size=100000

for table in observation_tables:
    lower_limit=1
    upper_limit=lower_limit+chunk_size-1 #sql BETWEEN is inclusive 
    query=f'SELECT MAX(ID) FROM {table}'
    
    mycursor.execute(query)
    for line in mycursor:
        number_of_rows=line[0]

    while lower_limit<=number_of_rows:
        query1='INSERT INTO observations '
        query2=f'(SELECT * FROM {table} WHERE ID BETWEEN {lower_limit} AND {upper_limit})'
        query=query1+query2
        mycursor.execute(query)
        lower_limit=upper_limit+1
        upper_limit=lower_limit+chunk_size-1

    print(f'{table} sucessfully inserted')

pfw_2016 sucessfully inserted
pfw_1988 sucessfully inserted
pfw_2001 sucessfully inserted
pfw_2021 sucessfully inserted
pfw_2006 sucessfully inserted
pfw_2011 sucessfully inserted
pfw_1996 sucessfully inserted


The next steps, cleaning and transformation of the actual data, are performed in a different file.